In [1]:
import numpy as np 
import pandas as pd 
from sklearn.cluster import KMeans
from NCPR_functions import load_NCPR

In [2]:
with open('data/similarity_matrix.txt', 'rb') as f:
    lines = f.readlines()

In [3]:
matrix = {}
matrix['protein_index'] = []
matrix['protein_name'] = []
matrix['data'] = []
for line in lines[6:]:
    strline = str(line)
    index = strline.index(':')
    protein_index = int(strline[index-1])
    protein_name = strline[index+2:].split(' ')[0]
    strdata = strline[index+4+len(protein_name):-3].split(' ')
    data = [float(a) for a in strdata if a != '']
    matrix['protein_index'].append(protein_index)
    matrix['protein_name'].append(protein_name)
    matrix['data'].append(data)

In [4]:
X = np.nan_to_num(np.array(matrix['data']))


In [5]:
matrix['labels'] = KMeans(n_clusters=400, random_state=0).fit_predict(X)

In [9]:
del matrix['data']
df = pd.DataFrame(matrix)
df.to_csv('data/clustered_NCPR.csv')

KeyError: 'data'

In [8]:
df = pd.read_csv("data/clustered_NCPR.csv")
cluster_df = df.groupby(['labels'])['protein_name'].count().sort_values(ascending=False).to_frame().reset_index()

In [9]:
cluster_df

,labels,protein_name
0,5,124
1,29,70
2,6,67
3,14,61
4,27,56
...,...,...
195,81,1
196,183,1
197,182,1
198,37,1


In [10]:
cluster_sizes = cluster_df['protein_name'].value_counts()

In [11]:
cluster_sizes

4      14
1      13
5      13
7      11
8      10
6       9
2       9
12      9
13      9
3       8
9       8
11      8
15      7
16      7
20      7
19      6
18      5
10      5
27      5
29      4
17      3
22      3
23      3
31      3
30      3
14      2
28      2
26      2
25      2
24      2
32      1
34      1
38      1
56      1
61      1
67      1
70      1
124     1
Name: protein_name, dtype: int64

In [12]:
dict_data, NCPR_df = load_NCPR('data/NCPR_bert.npz', 'data/uniprot-NCPR.tab', 'data/uniprot-NCPR.fasta')
by_species = NCPR_df.groupby(['species'])['index'].nunique().sort_values(ascending=False).to_frame().reset_index()

In [13]:
NCPR_df

,index,genus,species,ID,Sequence,species index,genus index
0,0,Colletotrichum,orchidophilum,A0A1G4AW05_9PEZI,MAELDTLDIVVLAVILLGTAAYFTKGKYWGVAKDPYANGFANANGA...,0,0
1,1,Neophocaena,asiaeorientalis,A0A341BHF7_NEOAA,MISFLINMGDSSVEAGATASDTVAEEVSLFSTTDMIMFSLIVGFLT...,1,1
2,2,Aspergillus,lacticoffeatus,A0A318ZYG3_ASPLB,MAQLDTLDLVVLAVLLVGSVAYFTKGTYWAVAKDPYASTGPAMNGA...,2,2
3,3,Heligmosomoides,polygyrus,A0A3P7X6Y1_HELPZ,MTPITPTVKPKDMSFIARMKNENRQVLIMYGSQTGTAEELSGRLAK...,3,3
4,4,Rhizopus,microsporus,A0A0A1PDJ3_RHIZD,MTRNNSHLLDTVDLILLGTIGLGTVAWFARHQITNRLFKSSSNGPQ...,4,4
...,...,...,...,...,...,...,...
2545,2545,Hesseltinella,vesiculosa,A0A1X2GQI1_9FUNG,MIVLGTVGLGLVAWLTRKVFFSKGTSSSSNYVSSPPPIATPNAPKK...,175,158
2546,2546,Pneumocystis,carinii,A0A0W4ZJ82_PNEC8,MFKEKQELTLDTFDIIFIFVILTTILYIFRGKFWTKNEYALSVSNN...,1272,610
2547,2547,Tabanus,bromius,A0A0K8TSW9_TABBR,DEPFLGPLDIVLLLTLIAGAAWWLFRNKKKEEVTTRAYAIQPTPIN...,1273,875
2548,2548,Cinnamomum,micranthum,A0A3S3MWQ3_9MAGN,MQSTTMKISPFDLTSAIVKRRFDPAAMASDSGATILAENLDFVMVL...,321,261


In [14]:
by_species

,species,index
0,padi,116
1,sp.,78
2,trutta,39
3,anshuiensis,26
4,grahami,26
...,...,...
1269,hapla,1
1270,haptotyla,1
1271,helianthi,1
1272,hemipterigena,1


In [15]:
single_labels = cluster_df.loc[cluster_df['protein_name']==1, 'labels'].values

In [19]:
counts = []
for label in single_labels:
    id = df.loc[df['labels']==label, 'protein_name'].values[0].split('|')[-1]
    species = NCPR_df.loc[NCPR_df['ID']==id, 'species'].values[0]
    print(species)
    species_count = by_species.loc[by_species['species']==species, 'index'].values[0]
    counts.append(species_count)

nasatum
clemensi
vison
prolifica
owczarzaki
reinhardtii
latifrons
solidus
prolifica
marinus
pneumoniae
hesperus
rosea


In [17]:
counts

[1, 1, 1, 9, 1, 1, 2, 2, 9, 2, 1, 4, 2]

In [18]:
by_species.loc[by_species['index']==9]

,species,index
20,prolifica,9
21,chinensis,9
22,murdjan,9
23,hucho,9
24,aurata,9
25,mays,9
26,salar,9
27,mustelinum,9
28,orbicularis,9
